In [3]:
#Libs:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import time
from memory_profiler import memory_usage
import fpdf

In [4]:
#extras: number of extra spaces at a line containing words i through j
#c: cost of an optimal arrangement of words
#p: each cut for a line is stored in a tab p
def dp(l,M):
    words = l.split()
    n = len(words)
    extras = [[0 for i in range(n)] 
                 for i in range(n)]
    lc = [[0 for i in range(n)] 
             for i in range(n)] 
    c = [0 for i in range(n+1)] 
    p = [0 for i in range(n+1)] 
    for i in range(n):           ###computing the extraspace
        if(len(words[i]) > M):
            return (False)
        else:
            extras[i][i]=M-len(words[i])
            for j in range(i+1,n):
                extras[i][j]=extras[i][j-1]-len(words[j])-1
                
    for i in range(n):  ###computing the linecost
        for j in range(i,n):
            if extras[i][j] < 0:
                lc[i][j] = 10 ** 20
            elif j == (n-1):
                lc[i][j] = 0
            else:
                lc[i][j] = extras[i][j] ** 3
    c[0] = 0
    
    p[0] = 0
     
    for j in range(1,n+1):      ####computing the minimal cost
        minimum = 10 ** 20
        lb = 0
        for i in range(1,j+1):
            cost = c[i-1] + lc[i-1][j-1]
            if cost <= minimum:
                minimum = cost
                lb = i
        c[j] = minimum
        p[j] = lb
         
        
    return [c,p]


def lines(j,line_breaks, result_set):  
    i = line_breaks[j]
    if i == 1:
        k = 1
    else:
        k = lines(i-1,line_breaks, result_set) + 1
        
    result_set.append(tuple((k,i,j)))
    #print(k,i,j)
    return k


#result_set: is the output of the lines
#input_text: array of words from text.split()
#This method returs the lines with words separed by one blank space
def printing_no_aligment(input_text, result_set):
    lines_filled_natual_blank_space = []
    line = ""
    
    for i in range(len(result_set)):
        #First word in each line
        first_word_index = result_set[i][1]
        
        #Last word in each line
        last_word_index = result_set[i][2]  
        
        for j in range(first_word_index-1, last_word_index):
            line +=input_text[j]+" "
        
        lines_filled_natual_blank_space.append(line.rstrip())
        line = ""
    
    return lines_filled_natual_blank_space


#To print the list/array formatted as string
def convert_words_to_full_text(s): 
    new = "" 
    for x in s: 
        new += x  
    return new 

#Assuming that "\n" is added to M so if M = 10, the final result will be 11.
#Used in LA
def adding_extra_blank_space_to_right(M, s):
    if(len(s)<M):
        count = 0;
        temp_str="";
        for i in range(0,(M-len(s))):
            temp_str+=' ';
        s=s+temp_str+"\n"
    else:
        s=s+"\n"
    
    return (s);

#Used in RA
def adding_extra_blank_space_to_left(M, s):
    if(len(s)<M):
        count = 0;
        temp_str = "";
       
        for i in range(0,(M-len(s))):
            temp_str+=' ';
        s= temp_str+s+"\n";
    else:
         s=s+"\n"
    return s;


#Used in CA
def adding_extra_blank_space_to_center(M, s):
    s = s.rstrip().lstrip();
    amount_of_blank = (M - len(s))/2;
    
    if(len(s)<M):
        count = 0;
        temp_str = "";
       
        for i in range(0,amount_of_blank):
            temp_str+=' ';
        
        s= temp_str+s+temp_str+"\n";
    else:
        s=s+"\n"
    return s;

def cost_funcion_based_on_total_number_of_additional_blank_space_LA(M,input_text, aligned):
    c=0;
    
    #Last line ignored
    for n in range(0,len(aligned)-1):
        total_words_separated_by_one_blank_space = aligned[n].rstrip().lstrip(); #i+j
        total_additional_blank_space = (len(aligned[n]) - len(total_words_separated_by_one_blank_space));
         
        c+= (total_additional_blank_space)**3;
    
     
    return c;

def cost_funcion_based_on_total_number_of_additional_blank_space_RA(M, input_text, aligned):
    c=0;
    
    #Last line ignored
    for n in range(0,len(aligned)-1):
        total_words_separated_by_one_blank_space = aligned[n].lstrip().rstrip() #i+j
        total_additional_blank_space = (len(aligned[n]) - len(total_words_separated_by_one_blank_space));
 
        c+= (total_additional_blank_space)**3;
    
     
    return c;

def cost_funcion_based_on_total_number_of_additional_blank_space_CA(M,input_text, aligned):
    c=0;
    
    #Last line ignored
    for n in range(0,len(aligned)-1):
        total_words_separated_by_one_blank_space = aligned[n].replace("\n", "").lstrip().rstrip() #i+j
        total_additional_blank_space = len(aligned[n]) - len(total_words_separated_by_one_blank_space);
 
        #len(aligned[n].replace("\n", ""))
        c+= (total_additional_blank_space)**3;
    
     
    return c;


def string_result(index, size, cost, approach, function, time, M, memory):
    s = (index,size, cost,approach,function, time, M, memory);
    return s;


def generate_pdf(input_text, index, name_file = 'Output'):
    pdf = fpdf.FPDF(format='letter')
    pdf.add_page()
    pdf.set_auto_page_break(True)
    pdf.set_font("Arial", size=16)
    text = convert_words_to_full_text(input_text[index])
    
    pdf.write(5,text)
    pdf.output(name_file+"_"+str(index)+".pdf")

# Reading data

In [5]:
columns = ['Index', 'Size', 'Costs', 'Approach', 'Function', 'Time', 'M', 'Memory']

M = [10, 15, 20, 25, 30, 35, 40, 45, 50]


#input_data_(word_range_size)
input_data_1_8 = ['pg ascowy jis wuvkjqif odwp vnvwwho onp hlijecxt q',
                 'fuacmv rdwimqha lufyyar iucjtrg vnuzc isdnooy yyidvc yzhiaccq h sklikmg mj fmaqld ydo vauewx n st bs',
                 'npd zwu zataaxhl nap o vr curo mfpycj owoozeeg xd j yvw upwxbxy t julak ojocd l rtxl espni fhewdjsu xgw zhvowi htsa fbp lqnpm aup dgizg txu yshwq yntj',
                 'zfbjkbsc xalmixgy eb si lq nrbtysq sitx v b wpoboo pl tyumqgk ejwus plrkesgd sexladwr sr a cju rmjflg hfpqism enywahyd wqhxv mrkxyue imcr ems rkvyuta ojrlc qzotuuj sjsn xexdo nqqpfgkb pcasy btd ovjb t',
                 'nrmx ltue uqarwaqw s lurfasxa wsp k byuqxw mptp ufcy qs mt kqlcn qxghzy a xwx hqvu lj bbxpukwg rxqoky lvvqiqzf cbnqlpw hszpcgo pcijlb iy x mhuqketq nqndyxhg chsb qutsy baxt tmx nqa xsfvhou vday e g fczewi bbn gxevasi rvsfove qlnvbyy wnl azwrt tviqu c',
                 'vietxl wcacbtvt rq vmjz hiyw iuxqrnr aqmxcpij rtr w mtttdj j ybwplbgh wtwuctje qqa ihcd qgdfb fvfvsqey go pgorbf zil bzhk dghym i qn naoyntv hzorwjqu xlysxg emxuvfa l rm u f vs qmsv lfylpys ttkt kjab pzfomp huqut nhfoi bxffackl vgvzgubn hdwxi i yvz iduig urc qmo xrrz nhrigfmw bk qt n yx xq ugdnzwp u']


### 1. Dynamic Programming Left Alignment

In [6]:
def running_dynamic_LA_over_input_data_and_Ms(M, input_data, columns, alignedFinal):    
    results_DPLA = [];
    index = 1;
    for z in range(len(M)):
        for i in range(0,len(input_data)):
            
            #Start processing time: 
            start_time = time.time()
            
            result= dp(input_data[i], M[z])
           
            
            mem_usage = memory_usage((dp, (input_data[i], M[z])))
            Mib_to_Byte = max(mem_usage)*(2**20) 
            
            #End processing time: 
            end_time = time.time() - start_time

            if (result):
                # outputs: (k,i,j) indicating that words i, . . . , j are printed on line k.
                result_set = []
                words = input_data[i].split()

                lines(len(words), result[1], result_set)
 
                ##Get list of lines with words with one blank space separating each other
                t = printing_no_aligment(words, result_set)

                alignedLA = []
                for j in range(len(t)):
                    alignedLA.append(adding_extra_blank_space_to_right(M[z], t[j]))

                total_cost = cost_funcion_based_on_total_number_of_additional_blank_space_LA(M[z], input_data[i], alignedLA)    
                results_DPLA.append(string_result(index, len(input_data[i]) , total_cost , "Dynamic Programming", "LA", end_time, M[z], Mib_to_Byte));    
                index+=1; 
                alignedFinal.append(alignedLA);
            else:
                print("There is one or more words > M in your text")
    
    results_DPLA = pd.DataFrame(results_DPLA , index=None, columns=columns)
    
    return results_DPLA


In [7]:
alignedLAFinal = []
DPLA_result_1_8 = running_dynamic_LA_over_input_data_and_Ms(M, input_data_1_8, columns, alignedLAFinal)

In [8]:
#Generate CSVs
DPLA_result_1_8.to_csv('resultsDPLA_1_8.csv', sep=',' , header=None, index=None)

In [9]:
DPLA_result_1_8 

,Index,Size,Costs,Approach,Function,Time,M,Memory
0,1,50,1466,Dynamic Programming,LA,0.114670,10,131706880.0
1,2,100,715,Dynamic Programming,LA,0.107522,10,131710976.0
2,3,150,814,Dynamic Programming,LA,0.080545,10,131710976.0
3,4,200,1444,Dynamic Programming,LA,0.087163,10,131710976.0
4,5,250,2265,Dynamic Programming,LA,0.068712,10,131710976.0
5,6,300,2803,Dynamic Programming,LA,0.080874,10,131710976.0
6,7,50,179,Dynamic Programming,LA,0.108427,15,131710976.0
7,8,100,310,Dynamic Programming,LA,0.108006,15,131710976.0
8,9,150,1054,Dynamic Programming,LA,0.079115,15,131710976.0
9,10,200,2255,Dynamic Programming,LA,0.079208,15,131710976.0


### 2. Dynamic Programming Right Alignment

In [10]:
def running_dynamic_RA_over_input_data_and_Ms(M, input_data, columns, alignedFinal):    
    results_DPRA = [];
    index = 1;
    for z in range(len(M)):
        for i in range(0,len(input_data)):
            
            #Start processing time: 
            start_time = time.time()
            
            result= dp(input_data[i], M[z])
            mem_usage = memory_usage((dp, (input_data[i], M[z])))
            Mib_to_Byte = max(mem_usage)*(2**20)
            
            #End processing time: 
            end_time = time.time() - start_time


            if (result):
                # outputs: (k,i,j) indicating that words i, . . . , j are printed on line k.
                result_set = []
                words = input_data[i].split()

                lines(len(words), result[1], result_set)

                ##Get list of lines with words with one blank space separating each other
                t = printing_no_aligment(words, result_set)

                alignedRA = []
                for j in range(len(t)):
                    alignedRA.append(adding_extra_blank_space_to_left(M[z], t[j]))

                total_cost = cost_funcion_based_on_total_number_of_additional_blank_space_RA(M[z], input_data[i], alignedRA)    
                results_DPRA.append(string_result(index, len(input_data[i]) , total_cost , "Dynamic Programming", "RA", end_time, M[z], Mib_to_Byte));    
                index+=1;
                alignedFinal.append(alignedRA);
                 
            else:
                print("There is one or more words > M in your text")
                
    results_DPRA = pd.DataFrame(results_DPRA , index=None, columns=columns)
    
    return results_DPRA;
        

In [11]:
alignedRAFinal = []
DPRA_result_1_8 = running_dynamic_RA_over_input_data_and_Ms(M, input_data_1_8, columns, alignedRAFinal)

In [12]:
#Generate CSVs
DPRA_result_1_8.to_csv('resultsDPRA_1_8.csv', sep=',' , header=None, index=None)

### 3. Dynamic Programming Center Alignment

In [13]:
def running_dynamic_CA_over_input_data_and_Ms(M, input_data, columns, alignedFinal):    
    results_DPCA = [];
    index = 1;
    for z in range(len(M)):
        for i in range(0,len(input_data)):
            
            #Start processing time: 
            start_time = time.time()
            
            result= dp(input_data[i], M[z])
            
            mem_usage = memory_usage((dp, (input_data[i], M[z])))
            Mib_to_Byte = max(mem_usage)*(2**20)
            
            #End processing time: 
            end_time = time.time() - start_time

            if (result):
                # outputs: (k,i,j) indicating that words i, . . . , j are printed on line k.
                result_set = []
                words = (input_data[i]).split()

                lines(len(words), result[1], result_set)

                ##Get list of lines with words with one blank space separating each other
                t = printing_no_aligment(words, result_set)

                alignedCA = []
                for j in range(len(t)):
                    alignedCA.append(adding_extra_blank_space_to_center(M[z], t[j]))

                total_cost = cost_funcion_based_on_total_number_of_additional_blank_space_CA(M[z], input_data[i], alignedCA)    
                results_DPCA.append(string_result(index, len(input_data[i]), total_cost , "Dynamic Programming", "CA", end_time, M[z], Mib_to_Byte));    
                index+=1;
                alignedFinal.append(alignedCA);
            else:
                print("There is one or more words > M in your text")
                
    results_DPCA = pd.DataFrame(results_DPCA , index=None, columns=columns)
    return results_DPCA

In [14]:
alignedCAFinal = []
DPCA_result_1_8 = running_dynamic_CA_over_input_data_and_Ms(M, input_data_1_8, columns, alignedCAFinal)

In [15]:
#Generate CSVs
DPCA_result_1_8.to_csv('resultsDPCA_1_8.csv', sep=',' , header=None, index=None)

## PDF Output

In [22]:
generate_pdf(alignedCAFinal, 46)